# Image scrapping

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlretrieve


def image_scrapping(url):
    for i in range(1,50):
      print(f"We are on page: {i}")
      url = urljoin(url,f"?page{i}")

      response = requests.get(url)

      if response.status_code != 200:
         print("There is error is getting the URL")
         return
            
      soup = BeautifulSoup(response.content,"html.parser")
      img_tags = soup.find_all("img")
            
      os.makedirs("scraped_images", exist_ok=True)
      
      for img_tag in img_tags:
          img_url = img_tag.get("src")
          if img_url:
             img_url = urljoin(url, img_url)
             img_name = os.path.basename(img_url)
             img_path = os.path.join("scraped_images", img_name)

          try:
             urlretrieve(img_url, img_path)
             print(f"Downloaded: {img_url}")
          except Exception as e:
             print(f"Failed to download: {img_url}")
             print(f"Error: {e}")    